In [1]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height: 999999999999 in;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [2]:
%matplotlib inline
import os
import os.path as osp

from IPython.display import display, Image, Markdown
from ipywidgets import interact, interactive
import ipywidgets as widgets 
import matplotlib.pylab as plt
import numpy as np
import pickle

from attackgraph import settings
import attackgraph.gambit_analysis as gambit_ops
import attackgraph.common.plot_ops as plot_ops

np.set_printoptions(precision=2)
RESULTS_DIR = settings.get_results_dir()

In [5]:
def display_result(run_name: str=""):
    """
    
    :param run_name: Name of the run.
    :type run_name: str
    """
    if run_name is None:
        return
    run_dir = osp.join(RESULTS_DIR, run_name)
    display(Markdown(f"# {osp.basename(run_dir)}"))
    display(run_dir)
    
    # Render the empirical game.
    display(Markdown("## Empirical Game\n * Row: Defender \n * Col: Attacker \n"))
    nfg_path = osp.join(run_dir, "payoff_matrix.nfg")
    payoff_defender, payoff_attacker = gambit_ops.read_normal_form_game_file(nfg_path)
    n_rows, n_cols = payoff_attacker.shape
    payoff_render = "| |" + "".join([f" {x} |" for x in range(n_cols)]) + "\n"
    payoff_render += "| " + "--- |"*(n_cols+1) + "\n"
    for row in range(n_cols):
        payoff_render += f"| {row} |"
        for col in range(n_rows):
            payoff_render += f" {payoff_defender[row][col]:.2f}, {payoff_attacker[row][col]:.2f} |"
        payoff_render += "\n"
    display(Markdown(payoff_render))
    
    # Print out each epoch of EGTA.
    for epoch in range(1, n_rows):
        display(Markdown(f"## Epoch {epoch}"))
        
        # Attacker.
        r_attacker = osp.join(run_dir, f"mean_rewards.attacker.{epoch+1}")
        try:
            if not osp.exists(r_attacker+".png"):
                plot_ops.generate_and_save_line_plot(
                    r_attacker+".pkl", 
                    f"Attacker {epoch}",
                    "Timestep",
                    "Average Reward")
            display(Image(r_attacker+".png"))
            plt.close()
        except:
            pass
        
        # Defender.
        r_defender = osp.join(run_dir, f"mean_rewards.defender.{epoch+1}")
        try:
            if not osp.exists(r_defender+".png"):
                plot_ops.generate_and_save_line_plot(
                    r_defender+".pkl", 
                    f"Defender {epoch}",
                    "Timestep",
                    "Average Reward")
            display(Image(r_defender+".png"))
            plt.close()
        except:
            pass
        
    
    

In [6]:
# Collect the names of every run result.
run_names = [x for x in os.listdir(RESULTS_DIR) if osp.isdir(osp.join(RESULTS_DIR, x))]
run_names = sorted(run_names)

run_name_widget = widgets.Dropdown(
    options=run_names,
    description="Run: ")

display(interactive(
    display_result,
    run_name=run_name_widget))

interactive(children=(Dropdown(description='Run: ', options=('08_11_dqn_state_enc', '08_11_gpu_dqn', '08_12_dq…